In [1]:
# import 
import pandas as pd
import yfinance as yf
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn 

In [3]:
train_df = pd.read_csv('train.csv')
train_df

,cov,var_SPY,mean_SPY,var_NVDA,mean_NVDA,rSPY_t-10,rNVDA_t-10,rSPY_t-9,rNVDA_t-9,rSPY_t-8,...,rSPY_t-5,rNVDA_t-5,rSPY_t-4,rNVDA_t-4,rSPY_t-3,rNVDA_t-3,rSPY_t-2,rNVDA_t-2,rSPY_t-1,rNVDA_t-1
0,0.000097,0.000059,0.001539,0.000331,-0.005720,0.002648,0.014602,0.000704,0.006397,0.004221,...,-0.009326,-0.033899,0.008446,0.013583,0.002705,-0.015634,-0.011224,-0.029495,0.012496,0.018702
1,0.000111,0.000093,0.000258,0.000296,-0.007581,0.000704,0.006397,0.004221,-0.019597,0.003327,...,0.008446,0.013583,0.002705,-0.015634,-0.011224,-0.029495,0.012496,0.018702,-0.010169,-0.004016
2,0.000153,0.000131,-0.001736,0.000327,-0.010007,0.004221,-0.019597,0.003327,0.002161,0.001397,...,0.002705,-0.015634,-0.011224,-0.029495,0.012496,0.018702,-0.010169,-0.004016,-0.019229,-0.017857
3,0.000184,0.000132,-0.004387,0.000395,-0.011507,0.003327,0.002161,0.001397,-0.014016,-0.009326,...,-0.011224,-0.029495,0.012496,0.018702,-0.010169,-0.004016,-0.019229,-0.017857,-0.022292,-0.034604
4,0.000175,0.000127,-0.004207,0.000424,-0.010022,0.001397,-0.014016,-0.009326,-0.033899,0.008446,...,0.012496,0.018702,-0.010169,-0.004016,-0.019229,-0.017857,-0.022292,-0.034604,0.005128,0.017011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,0.000073,0.000048,0.003544,0.000316,0.005178,0.004299,0.019530,0.003890,-0.018503,0.004567,...,-0.001646,0.011169,0.005625,0.024279,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270
3507,0.000072,0.000048,0.003315,0.000295,0.002899,0.003890,-0.018503,0.004567,0.022090,0.013790,...,0.005625,0.024279,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266
3508,0.000074,0.000049,0.003348,0.000242,0.005669,0.004567,0.022090,0.013790,0.009044,0.003209,...,0.006081,-0.009445,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266,0.004222,0.009195
3509,0.000072,0.000049,0.003072,0.000212,0.003740,0.013790,0.009044,0.003209,0.005448,-0.001646,...,-0.013857,-0.030098,0.009482,0.018270,0.002010,-0.003266,0.004222,0.009195,0.001808,0.002800


In [4]:
class LSTMdataset(Dataset):
    def __init__(self, dataframe):
        super().__init__()
        self.X = torch.tensor(dataframe.drop(['cov', 'var_NVDA', 'var_SPY'], axis=1).values, dtype=torch.float32) 
        self.y = torch.tensor(dataframe[['cov', 'var_NVDA', 'var_SPY']].values, dtype=torch.float32) 

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [5]:
train_dataset = LSTMdataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [6]:
# check shape in input
x, y = next(iter(train_loader))
print(x.shape, y.shape)
# need resize during training for lstm

torch.Size([32, 22]) torch.Size([32, 3])
